In [1]:
import csv
import importlib
import itertools
import numpy as np
import pandas as pd
import pickle
import os, shutil, time, h5py

from bct import *
importlib.reload(nbs)
from IPython.display import clear_output
from pathlib import Path
from sklearn import preprocessing
from scipy import stats

In [2]:
%%capture
#from tqdm import tqdm_notebook as tqdm
from tqdm.notebook import tqdm
tqdm().pandas()

In [3]:
# Setting Variables

## CSV Matrices of Connectivitiy Values as Z-scores (via cifti-connectivity)
pconncsv_dir = Path('/fast_scratch/jdr/ABCD/ABCD_3165_Pipe/pconns_working/pconn_csv')
datadir = Path('/fast_scratch/jdr/ABCD/ABCD_3165_Pipe/Data/IPV_Networks')
ROI_labels_path = Path('/fast_scratch/jdr/atlases/Schaefer400Tian4/Schaefer2018_400Parcels_7Networks_order_Tian_Subcortex_S4_label_reformatted.txt')
allArraysPath = Path('/fast_scratch/jdr/ABCD/ABCD_3165_Pipe/Data/IPV_Networks/allArrays.pkl')

In [4]:
allArrays = None
with open(allArraysPath, 'rb') as f:
    allArrays = pickle.load(f)

In [254]:
## Grab the ABCD file with the IPV exposure data and import to a table (skipping the header)
behav_df = pd.read_table('/fast_scratch/jdr/ABCD/ABCD_3165_Pipe/Data/abcd_ptsd01.txt', header=0, skiprows=[1])

## Subset to only include the baseline data
behav_df = behav_df[behav_df["eventname"] == "baseline_year_1_arm_1"]

## Drop unnecessary variables
behav_df = behav_df.drop(['collection_id', 'abcd_ptsd01_id', 'dataset_id', 
                      'src_subject_id', 'interview_date', 'eventname', 'collection_title'], axis=1)

## Reformat the 'subjectkey' variable to match the format in the CSV files
behav_df['subjectkey'] = 'sub-' + behav_df['subjectkey'].str.replace('_', '')

## In case they exist, drop the variables we're about to calculate
behav_df = behav_df.drop(['ExposureTotal', 'IPVTotal', 'ExposureAny', 'IPVAny'], axis=1, errors='ignore')

## Compute total overall exposure (all items) and total IPV exposure (IPV items only)
behav_df['ExposureTotal'] = behav_df.iloc[:, 3:19].sum(axis=1)
behav_df['IPVTotal'] = behav_df.iloc[:, 7:18].sum(axis=1)

## Binarize total and IPV exposure items 
behav_df['ExposureAny'] = np.where(behav_df['ExposureTotal'] == 0, 0, 1)
behav_df['IPVAny'] = np.where(behav_df['IPVTotal'] == 0, 0, 1)

## Dummy code sex variable to Sex01
behav_df['Sex01'] = np.where(behav_df['sex'] == 'M', 1, 0) # Must use numeric data as input (not M, F)

In [152]:
fcArrays_list = []
subjkey_list = []

for i in tqdm(list(range(len(allArrays)))):
    fcMatrix = allArrays[i][1].to_numpy()
    fcArray = fcMatrix[np.tril_indices(fcMatrix.shape[0], -1)]
    fcArrays_list.append(fcArray)
    
    subjkey = allArrays[i][0]
    subjkey_list.append(subjkey)
    
fcArrays = np.vstack(fcArrays_list)
subjkeys = np.vstack(subjkey_list)

  0%|          | 0/7220 [00:00<?, ?it/s]

In [ ]:
fcArrays_df = pd.DataFrame(fcArrays)
subjkeys_df = pd.DataFrame(subjkeys)
fc_df = pd.concat([subjkeys_df, fcArrays_df], axis=1, ignore_index=True)
fc_df=df.rename(columns = {0:'subjectkey'})

behav_df = behav_df.drop(behav_df.iloc[:,2:len(behav_df.columns)-1], axis = 1)
behav_df = behav_df.drop(behav_df.columns[[1]], axis=1)
comb_df = fc_df.merge(behav_df, on="subjectkey", how="left")

fcFeatures = comb_df.drop(comb_df.columns[[0,len(df.columns)]], axis=1)
sexLabels = comb_df.Sex01

In [259]:
#Splitting data into training and testing

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(fcFeatures, sexLabels, test_size=0.3)

In [262]:
#Creating the classification model
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

classifier = Sequential()
#First Hidden Layer
classifier.add(Dense(32, activation='tanh', kernel_initializer='random_normal', input_shape=fcFeatures.shape[1:]))
#Second  Hidden Layer
classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))
#Third  Hidden Layer
classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))
#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

In [267]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                3290624   
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 3,291,441
Trainable params: 3,291,441
Non-trainable params: 0
_________________________________________________________________


In [263]:
#Compiling the model
classifier.compile(optimizer = Adam(lr =.0001),loss='binary_crossentropy', metrics =['accuracy'])

In [264]:
#Fitting the model
classifier.fit(np.array(X_train),np.array(y_train), batch_size=32, epochs=100)

Epoch 1/100
158/158 [==============================] - 2s 6ms/step - loss: nan - accuracy: 0.4885
Epoch 2/100
158/158 [==============================] - 1s 6ms/step - loss: nan - accuracy: 0.5067
Epoch 3/100
158/158 [==============================] - 1s 6ms/step - loss: nan - accuracy: 0.5176
Epoch 4/100
158/158 [==============================] - 1s 7ms/step - loss: nan - accuracy: 0.5129
Epoch 5/100
158/158 [==============================] - 1s 9ms/step - loss: nan - accuracy: 0.4973
Epoch 6/100
158/158 [==============================] - 2s 10ms/step - loss: nan - accuracy: 0.5044
Epoch 7/100
158/158 [==============================] - 2s 10ms/step - loss: nan - accuracy: 0.5053
Epoch 8/100
158/158 [==============================] - 2s 10ms/step - loss: nan - accuracy: 0.5023
Epoch 9/100
158/158 [==============================] - 2s 12ms/step - loss: nan - accuracy: 0.5091
Epoch 10/100
158/158 [==============================] - 2s 12ms/step - loss: nan - accuracy: 0.5124
Epoch 11/100
1

In [265]:
#Determining accuracy on training
eval_model=classifier.evaluate(np.array(X_train), np.array(y_train))
eval_model

158/158 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.5042


[nan, 0.5041550993919373]